In [1]:
library(dplyr)
library(ggplot2)
library(PerformanceAnalytics)
library(lubridate)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: xts

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



######################### Warning from 'xts' package ##########################
#                                                                             #
# The dplyr lag() function breaks how base R's lag() function is supposed to  #
# work, which breaks lag(my_xts). Calls to lag(my_xts) that you type or       #
# source() into this session won't work correctly.                            #
#                                                                             #
# Use stats::lag() to make sure you're not using dplyr::lag(), or you can add #
# conflictRules('dplyr', exclude = 'lag') to your .Rprofile to

In [2]:
data = read.csv('compustat_food_bev.csv')


dpz_data = filter(data, tic == "DPZ")

dpz_data$datadate = as.Date(dpz_data$datadate, format = '%d/%m/%Y')
head(dpz_data)

,GVKEY,iid,datadate,tic,conm,cshtrd,prccd,prchd,prcld,prcod,exchg,sic
,<int>,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,160211,2,2020-09-01,DPZ,DOMINO'S PIZZA INC,502471,410.15,411.720,405.6500,408.44,11,5812
2,160211,2,2020-09-02,DPZ,DOMINO'S PIZZA INC,585397,403.60,413.180,402.6200,409.86,11,5812
3,160211,2,2020-09-03,DPZ,DOMINO'S PIZZA INC,799206,397.50,403.310,389.6400,403.00,11,5812
4,160211,2,2020-09-04,DPZ,DOMINO'S PIZZA INC,857808,381.81,401.200,376.3301,397.61,11,5812
5,160211,2,2020-09-08,DPZ,DOMINO'S PIZZA INC,682847,376.32,386.645,376.1100,381.57,11,5812
6,160211,2,2020-09-09,DPZ,DOMINO'S PIZZA INC,594344,382.07,387.690,380.4800,382.50,11,5812


In [ ]:
dpz_data$daily_return = (dpz_data$prccd / lag(dpz_data$prccd)) -1 

dpz_data$change_volume = dpz_data$cshtrd - lag(dpz_data$cshtrd)

dpz_data$change_open_close = dpz_data$prccd - dpz_data$prcod

dpz_data$MFV = (((dpz_data$prccd - dpz_data$prcld) - (dpz_data$prchd - dpz_data$prccd)) / (dpz_data$prchd - dpz_data$prcld)) * dpz_data$cshtrd

dpz_data_display = select(dpz_data, datadate, daily_return, change_volume, change_open_close, MFV)

head(dpz_data_display)

,datadate,daily_return,change_volume,change_open_close,MFV
,<date>,<dbl>,<int>,<dbl>,<dbl>
1,2020-09-01,NA,NA,1.71,242543.7
2,2020-09-02,-0.01596977,82926,-6.26,-476743.8
3,2020-09-03,-0.01511397,213809,-5.50,119851.7
4,2020-09-04,-0.03947170,58602,-15.80,-479784.6
5,2020-09-08,-0.01437888,-174961,-5.25,-655623.9
6,2020-09-09,0.01527955,-88503,-0.43,-332206.1


In [24]:
dpz_data = mutate(dpz_data, month = month(datadate), year  = year(datadate))
head(dpz_data)



,GVKEY,iid,datadate,tic,conm,cshtrd,prccd,prchd,prcld,prcod,exchg,sic,daily_return,change_volume,change_open_close,MFV,month,year
,<int>,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,160211,2,2020-09-01,DPZ,DOMINO'S PIZZA INC,502471,410.15,411.720,405.6500,408.44,11,5812,NA,NA,1.71,242543.7,9,2020
2,160211,2,2020-09-02,DPZ,DOMINO'S PIZZA INC,585397,403.60,413.180,402.6200,409.86,11,5812,-0.01596977,82926,-6.26,-476743.8,9,2020
3,160211,2,2020-09-03,DPZ,DOMINO'S PIZZA INC,799206,397.50,403.310,389.6400,403.00,11,5812,-0.01511397,213809,-5.50,119851.7,9,2020
4,160211,2,2020-09-04,DPZ,DOMINO'S PIZZA INC,857808,381.81,401.200,376.3301,397.61,11,5812,-0.03947170,58602,-15.80,-479784.6,9,2020
5,160211,2,2020-09-08,DPZ,DOMINO'S PIZZA INC,682847,376.32,386.645,376.1100,381.57,11,5812,-0.01437888,-174961,-5.25,-655623.9,9,2020
6,160211,2,2020-09-09,DPZ,DOMINO'S PIZZA INC,594344,382.07,387.690,380.4800,382.50,11,5812,0.01527955,-88503,-0.43,-332206.1,9,2020


In [29]:
dpz_june_2023 = filter(dpz_data, month == 9, year == 2023)
trade_vol = sum(dpz_june_2023$cshtrd, na.rm = TRUE)

print(trade_vol)

print(paste('The total trading value in June 2023 was' , trade_vol ))


[1] 2832257
[1] "The total trading value in June 2023 was 2832257"


In [ ]:

  
mean_daily_return = mean(dpz_data$daily_return, na.rm = TRUE)

print(mean_daily_return)

print(paste("The average daily return of Domino's Pizza was" , mean_daily_return ))


[1] 0.0001312795
[1] "The average daily return of Domino's pizza was 0.00013127951649099"


In [51]:
max_high = max(dpz_data$prchd, na.rm = TRUE)

print(max_closing)

print(paste("The highest price of Domino's Pizza was" , max_high ))


[1] 567.57
[1] "The highest price of Domino's Pizza was 567.57"


In [53]:


max_d_return = max(dpz_data$daily_return, na.rm = TRUE)
max_date = dpz_data$datadate[which.max(dpz_data$daily_return)]

print(max_d_return)
print(max_date)

print(paste("The maximum daily return of Domino's Pizza was" , max_d_return, 'on', max_date ))




[1] 0.1455237
[1] "2021-07-22"
[1] "The maximum daily return of Domino's Pizza was 0.145523736632864 on 2021-07-22"


On july 22, 2021, Domino's Pizza released their Q2 2021 earnings where they report revenue and profits more than expectations. Also on the same day they announced a $1 billion share repurchase program. These might be the reason why investors reacted positively which records the highest daily return for the given period. 